### Longest serving speakers of the house

#### Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
import altair as alt
import altair_grid as altgrid

alt.themes.register("grid", altgrid.theme)
alt.themes.enable("grid")

ThemeRegistry.enable('grid')

In [4]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.request import urlopen

In [5]:
import tabula
from tabula.io import read_pdf

In [6]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [7]:
pd.options.display.max_rows = 1000

### data from [house.gov](https://history.house.gov/People/Office/Speakers-List/)

In [8]:
df_gov = pd.read_html("https://history.house.gov/People/Office/Speakers-List/")[0]

In [9]:
df_gov.columns = df_gov.columns.str.lower().str.replace(" ", "_")

In [10]:
df_gov["term_start"] = pd.to_datetime(df_gov["date_elected"])

In [11]:
df_gov["term_end"] = pd.to_datetime(df_gov["date_elected"].shift(-1))

In [12]:
df_gov["days"] = df_gov["term_end"] - df_gov["term_start"]

In [13]:
speaker_tenures_gov = (
    df_gov.groupby("name")["days"]
    .sum()
    .reset_index()
    .sort_values("days", ascending=False)
)

### [wikipedia](https://en.wikipedia.org/wiki/List_of_speakers_of_the_United_States_House_of_Representatives#:~:text=Tip%20O'Neill%20had%20the,1869%2C%20he%20served%20one%20day.) data

In [14]:
df_wiki = pd.read_html(
    "https://en.wikipedia.org/wiki/List_of_speakers_of_the_United_States_House_of_Representatives#:~:text=Tip%20O'Neill%20had%20the,1869%2C%20he%20served%20one%20day."
)[1]

In [15]:
pattern = r"\[.*?\]"
df_wiki["term"] = df_wiki["Term"].str.replace(pattern, "", regex=True)

In [16]:
df_slim = df_wiki[["Name", "term"]][0:127].copy()

In [17]:
df_slim.columns = df_slim.columns.str.lower()

In [18]:
df_slim[["term_start", "term_end"]] = df_slim["term"].str.split(" – ", expand=True)

In [19]:
df_slim.at[43, "term_start"] = "March 3, 1869"
df_slim.at[126, "term_end"] = "January 3, 2023"

In [20]:
df_slim["term_start"] = pd.to_datetime(df_slim["term_start"])
df_slim["term_end"] = pd.to_datetime(df_slim["term_end"])

In [21]:
df_slim["length"] = df_slim["term_end"] - df_slim["term_start"]

In [22]:
speaker_tenures_wiki = df_slim.groupby("name")["length"].sum().reset_index().copy()

In [23]:
speaker_tenures_wiki.sort_values("length", ascending=False)

,name,length
44,Sam Rayburn,6263 days
50,Tip O'Neill,3637 days
22,John W. McCormack,3266 days
35,Nancy Pelosi,2918 days
5,Dennis Hastert,2915 days
2,Champ Clark,2551 days
9,Henry Clay,2428 days
13,James G. Blaine,2191 days
1,Carl Albert,2163 days
30,Joseph Gurney Cannon,2113 days


### can't merge these two datasets on name
#### tried by first day as speaker but only 50ish matched

In [24]:
start_date = df_slim.groupby("name")["term_start"].min().reset_index()

In [25]:
wiki_formerge = speaker_tenures_wiki.merge(start_date)

In [26]:
gov_startdate = df_gov.groupby("name")["term_start"].min().reset_index()

In [27]:
gov_formerge = speaker_tenures_gov.merge(gov_startdate)

In [28]:
merged_speaker_tenures = wiki_formerge.merge(gov_formerge, on="term_start", how="left")

In [29]:
merged_speaker_tenures = merged_speaker_tenures.rename(
    columns={
        "name_x": "wikiname",
        "name_y": "govname",
        "term_start": "firstterm_start",
        "length": "wiki_days",
        "days": "gov_days",
    }
)

In [30]:
merged_speaker_tenures

,wikiname,wiki_days,firstterm_start,govname,gov_days
0,Andrew Stevenson,1546 days,1827-12-03,"STEVENSON, Andrew",2191 days
1,Carl Albert,2163 days,1971-01-21,"ALBERT, Carl Bert",2175 days
2,Champ Clark,2551 days,1911-04-04,"CLARK, James Beauchamp (Champ)",2967 days
3,Charles Frederick Crisp,1026 days,1891-12-08,"CRISP, Charles Frederick",1455 days
4,David B. Henderson,912 days,1899-12-04,"HENDERSON, David Bremner",1435 days
5,Dennis Hastert,2915 days,1999-01-06,"HASTERT, John Dennis",2920 days
6,Frederick H. Gillett,1802 days,1919-05-19,"GILLETT, Frederick Huntington",2394 days
7,Frederick Muhlenberg,1159 days,1789-04-01,"MUHLENBERG, Frederick Augustus Conrad",1671 days
8,Galusha A. Grow,608 days,1861-07-04,"GROW, Galusha Aaron",886 days
9,Henry Clay,2428 days,1811-11-04,"CLAY, Henry",2030 days
